In [1]:
import numpy as np
import pandas as pd
import random as rand
import string as string

class Tape_Simulation:
    def __init__(self):
        self.clock = 0.0 #simulation clock
        
        self.num_in_q = 0 #tape in queue
        self.t_shuffle = 0 #time to shuffle tape in IBM system
        self.t_load = 0 #time to load tape
        self.total_wait_time = 0 #total wait time of each tape
        self.front_frame = 1
    
        self.num_in_system = 0
        self.number_in_queue = 0
        self.num_arrivals = 0 #number of arrivals
        self.num_departures = 0 #number of departures
        self.t_arrival = (-np.log(1-(np.random.uniform(low=0.0,high=1.0))) * 3) #time of next arrival sing Inverse Transform Sampling
        self.slowdown = rand.uniform(0.8,0.9) # slowdown added to speeds for reading and writing 
        
        self.state_S1 = 0 #status of server (binary)
        self.num_of_departures1 = 0
        self.dep_sum_S1 = 0 #total departure time of server
        self.t_departure_S1 = float('inf') #departure time from server
        self.data_stored_S1 = 1000; #default data amount stored in TBs
        self.TYPE_S1 = 'LTO8' #type of storage server
        if self.TYPE_S1 == 'LTO8':
            self.t_read_speed_S1 = 900*self.slowdown #speed to read tape - Set to a default value for now from Wikipedia in MB/s 900 if 8, 1000 if 9
        else:
            self.t_read_speed_S1 = 1000*self.slowdown #speed to read tape - Set to a default value for now from Wikipedia in MB/s 900 if 8, 1000 if 9
        
        self.state_S2 = 0 #status of server (binary)
        self.num_of_departures2 = 0
        self.dep_sum_S2 = 0 #total departure time of server
        self.t_departure_S2 = float('inf') #departure time from server
        self.data_stored_S2 = 1000; #default data amount stored in TBs
        self.TYPE_S2 = 'LTO8' #type of storage server
        if self.TYPE_S2 == 'LTO8':
            self.t_read_speed_S2 = 900*self.slowdown #speed to read tape - Set to a default value for now from Wikipedia in MB/s 900 if 8, 1000 if 9
        else:
            self.t_read_speed_S2 = 1000*self.slowdown #speed to read tape - Set to a default value for now from Wikipedia in MB/s 900 if 8, 1000 if 9
        
        self.state_S3 = 0 #status of server (binary)
        self.num_of_departures3 = 0
        self.dep_sum_S3 = 0 #total departure time of server
        self.t_departure_S3 = float('inf') #departure time from server
        self.data_stored_S3 = 1000; #default data amount stored in TBs
        self.TYPE_S3 = 'LTO8' #type of storage server
        if self.TYPE_S3 == 'LTO8':
            self.t_read_speed_S3 = 900*self.slowdown #speed to read tape - Set to a default value for now from Wikipedia in MB/s 900 if 8, 1000 if 9
        else:
            self.t_read_speed_S3 = 1000*self.slowdown #speed to read tape - Set to a default value for now from Wikipedia in MB/s 900 if 8, 1000 if 9
        
        self.state_S4 = 0 #status of server (binary)
        self.num_of_departures4 = 0
        self.dep_sum_S4 = 0 #total departure time of server
        self.t_departure_S4 = float('inf') #departure time from server
        self.data_stored_S4 = 1000; #default data amount stored in TBs
        self.TYPE_S4 = 'LTO9' #type of storage server
        if self.TYPE_S4 == 'LTO8':
            self.t_read_speed_S4 = 900*self.slowdown #speed to read tape - Set to a default value for now from Wikipedia in MB/s 900 if 8, 1000 if 9
        else:
            self.t_read_speed_S4 = 1000*self.slowdown #speed to read tape - Set to a default value for now from Wikipedia in MB/s 900 if 8, 1000 if 9
        
        self.state_S5 = 0 #status of server (binary)
        self.num_of_departures5 = 0
        self.dep_sum_S5 = 0 #total departure time of server
        self.t_departure_S5 = float('inf') #departure time from server
        self.data_stored_S5 = 1000; #default data amount stored in TBs
        self.TYPE_S5 = 'LTO9' #type of storage server
        if self.TYPE_S5 == 'LTO8':
            self.t_read_speed_S5 = 900*self.slowdown #speed to read tape - Set to a default value for now from Wikipedia in MB/s 900 if 8, 1000 if 9
        else:
            self.t_read_speed_S5 = 1000*self.slowdown #speed to read tape - Set to a default value for now from Wikipedia in MB/s 900 if 8, 1000 if 9
        
        # define tape properties
        self.num_tapes = 100
        self.assignment = [[0]*3 for m in range(self.num_tapes)]
        self.tape_address_len = 16 #length of address
        self.tape_addresses = ["" for x in range(self.num_tapes)] #array of empty addresses for tapes
        self.assign_tape_loc()
        feeder_address = self.tape_addresses[rand.randint(0,100)]
        self.search_sort()
        
    # simple search and sort algorithm
    def search_sort(self):
        feeder_address = self.tape_addresses[rand.randint(0,100)]
        for k in range(self.num_tapes):
            if feeder_address == self.tape_addresses[k]:
                location = self.assignment[k]
                if location[3] == 3:
                    if self.IBM_layer != 0 and self.IMB_layer != self.front_frame:
                        self.shuffle()
            else:
                location = -1
                self.t_shuffle = 0
        return location
    
    #assign location to all tapes
    def assign_tape_loc(self):
        for j in range(self.num_tapes):
            self.tape_addresses[j] = ''.join(rand.choices(string.ascii_letters + string.digits, k=self.tape_address_len)) #random alphanumeric addresses for tapes of specified length
            while True: #creates a location until it finds a location that is empty to assign
                self.server_assignment = rand.randint(1,5) #generates a random server to be assigned to for each tape
                self.server_tray_assignment = rand.randint(1,10)
                if self.server_assignment == 3: #IBM Server
                    self.IBM_layer = rand.randint(1,5)
                else:
                    self.IBM_layer = 0
                self.assignment[j] = [self.server_assignment, self.server_tray_assignment, self.IBM_layer]
                for count in range(j):
                    if self.assignment[j] == self.assignment[count]:
                        break
        return self.assignment
    
    #timing routine        
    def time_adv(self):
        t_next_event = min(self.t_departure_S1, self.t_departure_S2, self.t_departure_S3, self.t_departure_S4, self.t_departure_S5, self.t_arrival)
        self.total_wait_time += (self.num_in_q*(t_next_event-self.clock))
        self.clock = t_next_event
        
        if self.t_arrival < self.t_departure_S1 and self.t_arrival < self.t_departure_S2 and self.t_arrival < self.t_departure_S3 and self.t_arrival < self.t_departure_S4 and self.t_arrival < self.t_departure_S5:
            self.arrival()
        elif self.t_departure_S1 < self.t_arrival and self.t_departure_S1 < self.t_departure_S2 and self.t_departure_S1 < self.t_departure_S3 and self.t_departure_S1 < self.t_departure_S4 and self.t_departure_S1 < self.t_departure_S5:
            self.S1()
        elif self.t_departure_S2 < self.t_arrival and self.t_departure_S2 < self.t_departure_S1 and self.t_departure_S2 < self.t_departure_S3 and self.t_departure_S2 < self.t_departure_S4 and self.t_departure_S2 < self.t_departure_S5:
            self.S2()
        elif self.t_departure_S3 < self.t_arrival and self.t_departure_S3 < self.t_departure_S1 and self.t_departure_S3 < self.t_departure_S2 and self.t_departure_S3 < self.t_departure_S4 and self.t_departure_S3 < self.t_departure_S5:
            self.S3()
        elif self.t_departure_S4 < self.t_arrival and self.t_departure_S4 < self.t_departure_S1 and self.t_departure_S4 < self.t_departure_S2 and self.t_departure_S4 < self.t_departure_S3 and self.t_departure_S4 < self.t_departure_S5:
            self.S4()
        else:
            self.S5()
    
    def shuffle(self):
        if self.IBM_layer == self.front_frame+1 or self.IBM_layer == self.front_frame-1:
            self.t_shuffle = 1
        elif self.IBM_layer == self.front_frame+2 or self.IBM_layer == self.front_frame-2:
            self.t_shuffle = 2
        elif self.IBM_layer == self.front_frame+3 or self.IBM_layer == self.front_frame-3:
            self.t_shuffle = 3
        else:
            self.t_shuffle = 4
        self.front_frame = self.IBM_layer
        return self.t_shuffle
    
    #arrival event
    def arrival(self):
        self.num_arrivals += 1
        self.num_in_system += 1
        count = 0
        status_servers = [self.state_S1, self.state_S2, self.state_S3, self.state_S4, self.state_S5]
        if self.num_in_q == 0:
            if all(element == 1 for element in status_servers):
                self.num_in_q+=1
                self.number_in_queue+=1
                self.t_arrival = self.clock+self.gen_int_arr()
            elif all(element == 0 for element in status_servers):
                if rand.randint(1,5) == 1:
                    self.state_S1 = 1
                    self.dep1= self.gen_service_time_S1()
                    self.dep_sum_S1 += self.dep1
                    self.t_departure_S1 = self.clock + self.dep1
                    self.t_arrival = self.clock + (-np.log(1-(np.random.uniform(low=0.0,high=1.0))) * 3)
                elif rand.randint(1,5) == 2:
                    self.state_S2 = 1
                    self.dep2 = self.gen_service_time_S2()
                    self.dep_sum_S2 += self.dep2
                    self.t_departure_S2 = self.clock + self.dep2
                    self.t_arrival = self.clock + (-np.log(1-(np.random.uniform(low=0.0,high=1.0))) * 3)
                elif rand.randint(1,5) == 3:
                    self.state_S3 = 1
                    self.dep3 = self.gen_service_time_S3()
                    self.dep_sum_S3 += self.dep3
                    self.t_departure_S3 = self.clock + self.dep3
                    self.t_arrival = self.clock + (-np.log(1-(np.random.uniform(low=0.0,high=1.0))) * 3)
                elif rand.randint(1,5) == 4:
                    self.state_S4 = 1
                    self.dep4 = self.gen_service_time_S4()
                    self.dep_sum_S4 += self.dep4
                    self.t_departure_S4 = self.clock + self.dep4
                    self.t_arrival = self.clock + (-np.log(1-(np.random.uniform(low=0.0,high=1.0))) * 3)
                else:
                    self.state_S5 = 1
                    self.dep5 = self.gen_service_time_S5()
                    self.dep_sum_S5 += self.dep5
                    self.t_departure_S5 = self.clock + self.dep5
                    self.t_arrival = self.clock + (-np.log(1-(np.random.uniform(low=0.0,high=1.0))) * 3)
            else:
                for status in range(0,4):
                    if status_servers[status] == 1:
                        count+=1
                if count != 0:
                    seed = rand.randint(1,count)
                else:
                    seed = 0
                if seed == 1:
                    self.state_S1 = 1
                    self.dep1 = self.gen_service_time_S1()
                    self.dep_sum_S1 += self.dep1
                    self.t_departure_S1 = self.clock + self.dep1
                    self.t_arrival = self.clock + (-np.log(1-(np.random.uniform(low=0.0,high=1.0))) * 3)
                elif seed == 2:
                    self.state_S2 = 1
                    self.dep2 = self.gen_service_time_S2()
                    self.dep_sum_S2 += self.dep2
                    self.t_departure_S2 = self.clock + self.dep2
                    self.t_arrival = self.clock + (-np.log(1-(np.random.uniform(low=0.0,high=1.0))) * 3)
                elif seed == 3:
                    self.state_S3 = 1
                    self.dep3 = self.gen_service_time_S3()
                    self.dep_sum_S3 += self.dep3
                    self.t_departure_S3 = self.clock + self.dep3
                    self.t_arrival = self.clock + (-np.log(1-(np.random.uniform(low=0.0,high=1.0))) * 3)
                elif seed == 4:
                    self.state_S4 = 1
                    self.dep4 = self.gen_service_time_S4()
                    self.dep_sum_S4 += self.dep4
                    self.t_departure_S4 = self.clock + self.dep4
                    self.t_arrival = self.clock + (-np.log(1-(np.random.uniform(low=0.0,high=1.0))) * 3)
                else:
                    self.state_S5 = 1
                    self.dep5 = self.gen_service_time_S5()
                    self.dep_sum_S5 += self.dep5
                    self.t_departure_S5 = self.clock + self.dep5
                    self.t_arrival = self.clock + (-np.log(1-(np.random.uniform(low=0.0,high=1.0))) * 3)  
        else:
            self.num_in_q += 1
            self.number_in_queue += 1
            self.t_arrival = self.clock + (-np.log(1-(np.random.uniform(low=0.0,high=1.0))) * 3)
    def S1(self):
        self.num_of_departures1 += 1
        if self.num_in_q > 0:
            self.dep1 = self.gen_service_time_S1()
            self.dep_sum_S1 += self.dep1
            self.t_departure_S1 = self.clock + self.dep1
            self.num_in_q -= 1
        else:
            self.t_departure_S1 = float('inf')
            self.state_S1 = 0
    def S2(self):
        self.num_of_departures2 += 1
        if self.num_in_q > 0:
            self.dep2 = self.gen_service_time_S2()
            self.dep_sum_S2 += self.dep2
            self.t_departure_S2 = self.clock + self.dep2
            self.num_in_q -= 1
        else:
            self.t_departure_S2 = float('inf')
            self.state_S2 = 0
    def S3(self):
        self.num_of_departures3 += 1
        if self.num_in_q > 0:
            self.dep3 = self.gen_service_time_S3()
            self.dep_sum_S3 += self.dep3
            self.t_departure_S3 = self.clock + self.dep3
            self.num_in_q -= 1
        else:
            self.t_departure_S3 = float('inf')
            self.state_S3 = 0
    def S4(self):
        self.num_of_departures4 += 1
        if self.num_in_q > 0:
            self.dep4 = self.gen_service_time_S4()
            self.dep_sum_S4 += self.dep4
            self.t_departure_S4 = self.clock + self.dep4
            self.num_in_q -= 1
        else:
            self.t_departure_S4 = float('inf')
            self.state_S4 = 0
    def S5(self):
        self.num_of_departures5 += 1
        if self.num_in_q > 0:
            self.dep5 = self.gen_service_time_S5()
            self.dep_sum_S5 += self.dep5
            self.t_departure_S5 = self.clock + self.dep5
            self.num_in_q -= 1
        else:
            self.t_departure_S5 = float('inf')
            self.state_S5 = 0
    
    #Generate Arrival and Service Time from Servers
    def gen_service_time_S1(self):
        if self.TYPE_S1 == 'LTO8':
            load_to_ready_time = 15 #in sec
            rewind_time = 59
            other_mount_activity = self.data_stored_S1/self.t_read_speed_S1 #in seconds
            return (load_to_ready_time+rewind_time+other_mount_activity+self.t_shuffle)
        else:
            load_to_ready_time = 17
            rewind_time = 55
            other_mount_activity = self.data_stored_S1/self.t_read_speed_S1 #in seconds
            return (load_to_ready_time+rewind_time+other_mount_activity+self.t_shuffle)
    def gen_service_time_S2(self):
        if self.TYPE_S2 == 'LTO8':
            load_to_ready_time = 15
            rewind_time = 59
            other_mount_activity = self.data_stored_S2/self.t_read_speed_S2
            return (load_to_ready_time+rewind_time+other_mount_activity+self.t_shuffle)
        else:
            load_to_ready_time = 17
            rewind_time = 55
            other_mount_activity = self.data_stored_S2/self.t_read_speed_S2
            return (load_to_ready_time+rewind_time+other_mount_activity+self.t_shuffle)
    def gen_service_time_S3(self):
        if self.TYPE_S3 == 'LTO8':
            load_to_ready_time = 15
            rewind_time = 59
            other_mount_activity = self.data_stored_S3/self.t_read_speed_S3
            return (load_to_ready_time+rewind_time+other_mount_activity+self.t_shuffle)
        else:
            load_to_ready_time = 17
            rewind_time = 55
            other_mount_activity = self.data_stored_S3/self.t_read_speed_S3
            return (load_to_ready_time+rewind_time+other_mount_activity+self.t_shuffle)
    def gen_service_time_S4(self):
        if self.TYPE_S4 == 'LTO8':
            load_to_ready_time = 15
            rewind_time = 59
            other_mount_activity = self.data_stored_S4/self.t_read_speed_S4
            return (load_to_ready_time+rewind_time+other_mount_activity+self.t_shuffle)
        else:
            load_to_ready_time = 17
            rewind_time = 55
            other_mount_activity = self.data_stored_S4/self.t_read_speed_S4
            return (load_to_ready_time+rewind_time+other_mount_activity+self.t_shuffle)
    def gen_service_time_S5(self):
        if self.TYPE_S5 == 'LTO8':
            load_to_ready_time = 15
            rewind_time = 59
            other_mount_activity = self.data_stored_S5/self.t_read_speed_S5
            return (load_to_ready_time+rewind_time+other_mount_activity+self.t_shuffle)
        else:
            load_to_ready_time = 17
            rewind_time = 59
            other_mount_activity = self.data_stored_S5/self.t_read_speed_S5
            return (load_to_ready_time+rewind_time+other_mount_activity+self.t_shuffle)

# CREATE INSTANCE of TIME_SIMULATION OBJECT with data stored in provided
server = Tape_Simulation()
# RUN SIMULATION
df = pd.DataFrame(columns = ['Average interarrival time','Average service time Server 1','Average service time Server 2','Average service time Server 3','Average service time Server 4','Average service time Server 5','Utilization Server 1','Utilization Server 2','Utilization Server 3','Utilization Server 4','Utilization Server 5','Total Data Queued in Line Server','Total average wait time Server'])

for i in range(100):
    np.random.seed(i)
    server.__init__()
    
    while server.clock <= 240:
        server.time_adv()
    if server.num_arrivals != 0:
        avg_inter_time = server.clock/server.num_arrivals
    else:
        avg_inter_time = -1 #invalid time indication
    if server.num_departures != 0:
        service_time_s1 = server.dep_sum_S1/server.num_departures
        service_time_s2 = server.dep_sum_S2/server.num_departures
        service_time_s3 = server.dep_sum_S3/server.num_departures
        service_time_s4 = server.dep_sum_S4/server.num_departures
        service_time_s5 = server.dep_sum_S5/server.num_departures
    else:
        service_time_s1 = -1
        service_time_s2 = -1
        service_time_s3 = -1
        service_time_s4 = -1
        service_time_s5 = -1
    if server.clock != 0:
        util_time_s1 = server.dep_sum_S1/server.clock
        util_time_s2 = server.dep_sum_S2/server.clock
        util_time_s3 = server.dep_sum_S3/server.clock
        util_time_s4 = server.dep_sum_S4/server.clock
        util_time_s5 = server.dep_sum_S5/server.clock
    else:
        util_time_s1 = -1
        util_time_s2 = -1
        util_time_s3 = -1
        util_time_s4 = -1
        util_time_s5 = -1
        
    a = pd.Series([avg_inter_time,service_time_s1,service_time_s2,service_time_s3,service_time_s4,service_time_s5,util_time_s1,util_time_s2,util_time_s3,util_time_s4,util_time_s5,server.number_in_queue,server.total_wait_time],index = df.columns)
    df = df.append(a,ignore_index = True)
    
df.to_excel('Results.xlsx') 

KeyboardInterrupt: 